<a href="https://colab.research.google.com/github/Annieng184/Annieng184/blob/main/SQL_Query_practice_with_Olist_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Query

![](https://storage.googleapis.com/kaggle-datasets-images/55151/105464/d59245a7014a35a35cc7f7b721de4dae/dataset-cover.png?t=2018-09-21-16-21-21)

You will be given a dataset on BigQuery about __Brazillian E-commerce__ (Source: [Kaggle](https://www.kaggle.com/olistbr/brazilian-ecommerce)). 

Your task is to __write SQL queries to answer the three given questions__.


![](https://drive.google.com/uc?export=view&id=1nuLGF7VnRJmMZq7BkS1hPFLOZTxGd-UA)

In [ ]:
# show all the table names
query = '''
SELECT name
FROM sqlite_master 
    WHERE type ='table' 
      AND name NOT LIKE 'sqlite_%'
'''

pd.read_sql_query(query, conn)

,name
0,customers
1,sellers
2,products
3,order_reviews
4,order_payments
5,order_items
6,catergory_name_translation
7,geo_location
8,orders


### Question 1: Find top 5 best-selling product categories

In [ ]:
query = '''
WITH tmp as (
  SELECT *
  FROM order_items i 
  LEFT JOIN products p on i.product_id = p.product_id
  LEFT JOIN catergory_name_translation C on p.product_category_name = c.category_name)


SELECT category_name_english, count(order_id) as number_sold
FROM tmp
GROUP BY category_name_english
ORDER BY number_sold desc
LIMIT 5
'''

pd.read_sql_query(query, conn)

,category_name_english,number_sold
0,bed_bath_table,11115
1,health_beauty,9670
2,sports_leisure,8641
3,furniture_decor,8334
4,computers_accessories,7827


### Question 2: Statistic the revenue of all sellers by month. 

> Hint: orders are marked on a month when it is approved at that month

> Hint: some orders does not have approved day - they are represented as empty strings

In [ ]:
query = '''
WITH tmp as (
  SELECT *
  FROM order_items i 
    JOIN sellers s on i.seller_id = s.seller_id
    JOIN orders o on o.order_id = i.order_id
    WHERE order_approved_at != ''),

tmp2 as (
  SELECT *, strftime('%m/%Y', order_approved_at) as month
  FROM tmp)

SELECT month, seller_id, (price + freight_value) as revenue
FROM tmp2
GROUP BY month, seller_id
ORDER BY month, revenue desc
'''


pd.read_sql_query(query, conn)

,month,seller_id,revenue
0,01/2017,403aa86912527d730337ffcb0fb096ab,2217.99
1,01/2017,1d0997ff06b524ce9289ffd75114ecd3,2023.00
2,01/2017,8b8cfc8305aa441e4239358c9f6f2485,1961.00
3,01/2017,7e93a43ef30c4f03f38b393420bc753a,1326.79
4,01/2017,5dceca129747e92ff8ef7a997dc4f8ca,893.00
...,...,...,...
16437,12/2017,fc38b5dceee1a730fad8853453437fbd,19.48
16438,12/2017,e84ad2127668df3aafc6b73531a0beb8,16.34
16439,12/2017,077c5fae4bea9500e3737b16f71b9d3a,14.67
16440,12/2017,e6a69c4a27dfdd98ffe5aa757ad744bc,13.82


### Question 3: Statistics the seller's information according to the table below. 

Where:

```
number_of_product: the total of distinct products were sold (not the amount of product was sold)
product_category_revenue: the revenue of a product category of a seller
rank_in_category: rank by revenue of a product category of a seller 
total_revenue: the total revenue of a seller
```

In [ ]:
query = '''
WITH tmp as (
  SELECT *
  FROM order_items i 
    LEFT JOIN sellers s on i.seller_id = s.seller_id
    LEFT JOIN orders o on o.order_id = i.order_id
    LEFT JOIN products p on p.product_id = i.product_id
    LEFT JOIN catergory_name_translation C on p.product_category_name = c.category_name
    WHERE order_approved_at != '')

SELECT seller_id, sum(price + freight_value) as total_revenue, count(distinct(product_id)) as number_of_products
FROM tmp
GROUP BY seller_id
ORDER BY total_revenue desc
'''


pd.read_sql_query(query, conn)

,seller_id,total_revenue,number_of_products
0,4869f7a5dfa277a7dca6462dcf3b52b2,249640.70,95
1,7c67e1448b00f6e969d365cea6b010ab,239536.44,198
2,53243585a1d6dc2643021fd1853d8905,235856.68,23
3,4a3ca9315b744ce9f8e9374361493884,235539.96,399
4,fa1c13f2614d7b5c4749cbc52fecda94,204084.73,289
...,...,...,...
3090,702835e4b785b67a084280efca355756,18.56,1
3091,4965a7002cca77301c82d3f91b82e1a9,16.36,1
3092,1fa2d3def6adfa70e58c276bb64fe5bb,15.90,1
3093,77128dec4bec4878c37ab7d6169d6f26,15.22,1


In [ ]:
query = '''
WITH tmp as (
  SELECT *
  FROM order_items i 
    JOIN sellers s on i.seller_id = s.seller_id
    JOIN orders o on o.order_id = i.order_id
    JOIN products p on p.product_id = i.product_id
    JOIN catergory_name_translation C on p.product_category_name = c.category_name
    WHERE order_approved_at != ''),
    
tmp2 as (
  SELECT seller_id, category_name_english, sum(price + freight_value) as product_category_revenue
  FROM tmp
  GROUP BY seller_id, category_name_english
  ORDER BY category_name_english, product_category_revenue desc)

SELECT *, RANK() OVER(PARTITION BY category_name_english ORDER BY product_category_revenue desc) rank_in_category
FROM tmp2

'''


pd.read_sql_query(query, conn)

,seller_id,category_name_english,product_category_revenue,rank_in_category
0,e59aa562b9f8076dd550fcddf0e73491,agro_industry_and_commerce,33803.97,1
1,6bd69102ab48df500790a8cecfc285c2,agro_industry_and_commerce,8359.02,2
2,f08a5b9dd6767129688d001acafc21e5,agro_industry_and_commerce,7796.21,3
3,2528744c5ef5d955adc318720a94d2e7,agro_industry_and_commerce,6508.70,4
4,31ae0774c17fabd06ff707cc5bde005f,agro_industry_and_commerce,5690.23,5
...,...,...,...,...
6346,2ff97219cb8622eaf3cd89b7d9c09824,watches_gifts,56.22,97
6347,891071be6ba827b591264c90c2ae8a63,watches_gifts,35.96,98
6348,b98b715fe9e276204e2a3464f48cd796,watches_gifts,33.71,99
6349,1da3aeb70d7989d1e6d9b0e887f97c23,watches_gifts,27.22,100


In [ ]:
query = '''
WITH tmp as (
  SELECT *
  FROM order_items i 
    JOIN sellers s on i.seller_id = s.seller_id
    JOIN orders o on o.order_id = i.order_id
    JOIN products p on p.product_id = i.product_id
    JOIN catergory_name_translation C on p.product_category_name = c.category_name
    WHERE order_approved_at != ''),
    
tmp2 as (
  SELECT seller_id, sum(price + freight_value) as total_revenue, count(distinct(product_id)) as number_of_products
  FROM tmp
  GROUP BY seller_id
  ORDER BY total_revenue desc),

tmp3 as (
  SELECT seller_id, category_name_english as category_name, sum(price + freight_value) as product_category_revenue
  FROM tmp
  GROUP BY seller_id, category_name
  ORDER BY category_name, product_category_revenue desc),
  
tmp4 as (
  SELECT *, RANK() OVER(PARTITION BY category_name ORDER BY product_category_revenue desc) rank_in_category
  FROM tmp3)

SELECT *
FROM tmp4 LEFT JOIN tmp2 USING(seller_id)
'''


pd.read_sql_query(query, conn)

,seller_id,category_name,product_category_revenue,rank_in_category,total_revenue,number_of_products
0,e59aa562b9f8076dd550fcddf0e73491,agro_industry_and_commerce,33803.97,1,33803.97,9
1,6bd69102ab48df500790a8cecfc285c2,agro_industry_and_commerce,8359.02,2,8359.02,3
2,f08a5b9dd6767129688d001acafc21e5,agro_industry_and_commerce,7796.21,3,14565.74,7
3,2528744c5ef5d955adc318720a94d2e7,agro_industry_and_commerce,6508.70,4,6508.70,3
4,31ae0774c17fabd06ff707cc5bde005f,agro_industry_and_commerce,5690.23,5,7570.67,9
...,...,...,...,...,...,...
6346,2ff97219cb8622eaf3cd89b7d9c09824,watches_gifts,56.22,97,5866.03,45
6347,891071be6ba827b591264c90c2ae8a63,watches_gifts,35.96,98,5454.35,31
6348,b98b715fe9e276204e2a3464f48cd796,watches_gifts,33.71,99,517.62,4
6349,1da3aeb70d7989d1e6d9b0e887f97c23,watches_gifts,27.22,100,13055.36,155
